In [ ]:
import glob
import os

from addict import Dict
import functools
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import vmodel.util.xarray as xrutil
from vmodel import plot
from vmodel import metrics

%load_ext autoreload
%autoreload 2

In [ ]:
plt.rcParams['figure.autolayout'] = True
plt.rcParams['figure.figsize'] = [6, 3.5]

In [ ]:
# Each subfolder contains an experiment
expdir = '/home/fabian/vmodel_datasets/ref_distance_figure'
plotname = expdir.split(os.path.sep)[-1]
plotname

In [ ]:
paths = sorted(glob.glob(f'{expdir}/**/*merged*.nc', recursive=True))
paths[0], len(paths)

In [ ]:
# Fill experimental dictionary: experiment -> num_agents -> ds
expdict = Dict()
key = 'num_agents'
for path in paths:
    # ds = pickle.load(open(path, 'rb'))
    ds = xr.open_dataset(path)
    *_, expname, fname = path.split(os.path.sep)
    expdict[float(expname.replace('dist', ''))] = ds

In [ ]:
ds = xr.concat(expdict.values(), 'dist')
ds.coords['dist'] = list(expdict.keys())

In [ ]:
sepgains = [d ** 2 / 2 for d in ds.dist.data]
sepgains

In [ ]:
fig, ax = plt.subplots()
mindist = ds.nndist.min('agent').isel(time=slice(400, None)).mean('time')
for dist, dsagents in mindist.groupby('dist'):
    xs = dsagents.nagents.data
    ys, yerrs = dsagents.mean('run'), dsagents.std('run')
    ax.errorbar(xs, ys, yerrs, fmt='-o', capsize=3)
ax.grid()
# ax.legend()
ax.set(xscale='log')
ax.set(xticks=ds.nagents.data, xticklabels=ds.nagents.data)
ax.set(yticks=ds.dist.data, yticklabels=ds.dist.data)
# ax.set(ylim=(0, None))
secax = ax.secondary_yaxis('right')
ax.set(xlabel=r'number of agents $N$', ylabel=r'min. distance $d^\mathrm{min}$ [$m$]')
secax.set(ylabel=r'separation gain $k^\mathrm{sep}$ [$m s^{-1}$]')
secax.set(yticks=ds.dist.data, yticklabels=sepgains)
pass
fig.savefig(f'ref_distance_scalability.pdf', bbox_inches='tight')